# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


**Question 1**

In [2]:
limit = 5
generator = square_root_generator(limit)

sum = 0
for sqrt_value in generator:
    sum = sum + sqrt_value
print(sum)

8.382332347441762


**Question 2**

In [3]:
limit = 13
generator = square_root_generator(limit)
print(list(generator)[12])

3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [4]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [5]:
import dlt

generators_pipeline = dlt.pipeline(
    destination="duckdb", dataset_name="generators")
info = generators_pipeline.run(
    people_1(), table_name="question_3", write_disposition="append"
)
print(info)
info = generators_pipeline.run(
    people_2(), table_name="question_3", write_disposition="append"
)
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.39 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////home/ubuntu/projects/de_zoomcamp/workshop-1-data-ingestion/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708224066.3477793 is LOADED and contains no failed jobs
Pipeline dlt_ipykernel_launcher load step completed in 0.32 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////home/ubuntu/projects/de_zoomcamp/workshop-1-data-ingestion/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708224067.1726687 is LOADED and contains no failed jobs


In [6]:
import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print("Tables:")
display(conn.sql("show tables"))

Tables:


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ question_3          │
└─────────────────────┘

**Question 3**

In [7]:
age_3 = conn.sql("SELECT SUM(age) FROM question_3").df()
display(age_3)

,sum(age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [8]:
# to do: homework :)
pipeline = dlt.pipeline(destination="duckdb", dataset_name="question_4")
info = generators_pipeline.run(
    people_1(),
    table_name="question_4",
    write_disposition="merge",
    primary_key="id",
)
print(info)
info = generators_pipeline.run(
    people_2(),
    table_name="question_4",
    write_disposition="merge",
    primary_key="id",
)
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.45 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////home/ubuntu/projects/de_zoomcamp/workshop-1-data-ingestion/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708224068.0135312 is LOADED and contains no failed jobs
Pipeline dlt_ipykernel_launcher load step completed in 0.42 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////home/ubuntu/projects/de_zoomcamp/workshop-1-data-ingestion/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708224068.8867853 is LOADED and contains no failed jobs


In [9]:
classmethodonn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print("Tables: ")
display(conn.sql("show tables"))

Tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ question_3          │
│ question_4          │
└─────────────────────┘

**Question 4**

In [10]:
people = conn.sql("SELECT * FROM question_4").df()
display(people)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,2,Person_2,27,City_A,1708224068.0135312,veSZnALyFmrIbg,None
1,1,Person_1,26,City_A,1708224068.0135312,bzEc/Ezq+2Pk7A,None
2,8,Person_8,38,City_B,1708224068.8867853,mKA8iY2pWAaEqA,Job_8
3,4,Person_4,34,City_B,1708224068.8867853,0bNZVQ1m57ENAg,Job_4
4,7,Person_7,37,City_B,1708224068.8867853,kAKESWivsNxwCg,Job_7
5,5,Person_5,35,City_B,1708224068.8867853,D1ZMqMK6Q8qQHA,Job_5
6,3,Person_3,33,City_B,1708224068.8867853,rH5FRS+0PGyQhQ,Job_3
7,6,Person_6,36,City_B,1708224068.8867853,2JjgXXnn/5+W1A,Job_6


In [11]:
age_4 = conn.sql("SELECT SUM(age) FROM question_4").df()
display(age_4)

,sum(age)
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX